In [1]:
import geopandas as gpd
import folium
from shapely.geometry import Point
import pandas as pd
import numpy as np

In [12]:
from shapely.geometry import Point

In [13]:
# 读取文件
boundary_gdf = gpd.read_file('Seattle_City_Boundary.geojson')
metro_gdf = gpd.read_file('King_County_Metro_Transit_Routes.geojson')

In [14]:
boundary_gdf.head(3)

,OBJECTID,GEOID20,GROSS_ACRES,LAND_ACRES,WATER_ACRES,NAME,TRACT_NUMB,BASENAME,UVDA_AREA,CRA_NO,CRA_GRP,GEN_ALIAS,DETL_NAMES,C_DISTRICT,geometry
0,1,53033000402,444.493643,444.527261,0.000000,Census Tract 4.02,402,4.02,Bitter Lake Village,9.1,9,Broadview/Bitter Lake,"Broadview, Bitter Lake",5,"POLYGON ((1267598.433 267477.138, 1267788.924 ..."
1,2,53033000403,180.930716,165.408389,15.535762,Census Tract 4.03,403,4.03,Bitter Lake Village,9.1,9,Broadview/Bitter Lake,"Broadview, Bitter Lake",5,"POLYGON ((1265848.993 271492.767, 1265842.812 ..."
2,3,53033000700,319.838996,319.862596,0.000000,Census Tract 7,700,7,Lake City,8.2,8,Olympic Hills/Victory Heights,"Lake City, Victory Heights, Olympic Hills, Pin...",5,"POLYGON ((1277629.018 265876.292, 1277833.975 ..."


In [15]:
metro_gdf.head(3)

,OBJECTID_1,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
0,1,1,155,4,IN SERVICE,Y,100111,L,22,62910.364200,"MULTILINESTRING ((-122.36591 47.51736, -122.36..."
1,2,2,155,4,IN SERVICE,Y,100113,L,221,129813.753086,"MULTILINESTRING ((-122.144 47.57967, -122.1456..."
2,3,3,155,4,IN SERVICE,Y,100132,L,24,81678.853571,"MULTILINESTRING ((-122.32905 47.59334, -122.32..."


In [24]:

# 确定相同crs
metro_gdf = metro_gdf.to_crs('EPSG:4326')

if boundary_gdf.crs != metro_gdf.crs:
    boundary_gdf = boundary_gdf.to_crs('EPSG:4326')

# 裁剪数据
metro_gdf_clipped = boundary_gdf.clip(metro_gdf)

# 定义样式函数
def style_function(feature):
    return{
        'color':'orange',
        'weight':3,
        'opacity':0.8
    }

# 创建 Folium 地图对象
m = folium.Map(location=[47.6062, -122.3321], zoom_start=10)

# 添加一个TileLayer图层
folium.TileLayer(tiles ='cartodbpositron').add_to(m)

# 将裁剪后的 GeoDataFrame 转换为 GeoJSON 并添加到地图上
folium.GeoJson(metro_gdf_clipped,
               style_function=style_function
               ).add_to(m)

m.save('Metro_Seattle.html')
m

In [25]:
### 计算中心坐标
from shapely.geometry import Point

# 读取城市边界数据
boundary_gdf = gpd.read_file('Seattle_City_Boundary.geojson')

# 读取 Metro 路线数据
metro_gdf = gpd.read_file('King_County_Metro_Transit_Routes.geojson')

# 确保边界和数据使用相同的坐标系统
if metro_gdf.crs != boundary_gdf.crs:
    metro_gdf = metro_gdf.to_crs(boundary_gdf.crs)

# 裁剪数据，使其仅包含边界内的部分
gdf_clipped = boundary_gdf.clip(metro_gdf)

# 投影到适当的投影坐标系统（例如 EPSG:3857）
if gdf_clipped.crs != 'EPSG:4326':
    gdf_clipped = gdf_clipped.to_crs(epsg=3857)

# 计算几何中心
centroid = gdf_clipped.geometry.centroid
center_x = centroid.x.mean()
center_y = centroid.y.mean()

# 将计算出的中心点转换回 WGS84 坐标系统
center_point = gpd.GeoSeries([Point(center_x, center_y)], crs='EPSG:3857').to_crs(epsg=4326).geometry.iloc[0]
center_location = [center_point.y, center_point.x]

# 定义样式函数
def style_function(feature):
    return {
        'color': 'blue',    # 线条颜色
        'weight': 3,        # 线条粗细
        'opacity': 0.7      # 线条透明度
    }

# 初始化 Folium 地图
m = folium.Map(location=center_location, zoom_start=12)

# 添加一个TileLayer图层
folium.TileLayer(tiles ='cartodbpositron').add_to(m)

# 将裁剪后的 GeoDataFrame 转换为 GeoJSON 并添加到地图上
folium.GeoJson(gdf_clipped, style_function=style_function).add_to(m)

# 保存或展示地图
m.save('Metro_Seattle_centroid.html')
m